In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [12]:
X, y = make_regression(
    n_samples=1000,
    n_features=8,
    n_informative=5,
    noise=15,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))


MSE: 232.850738847647
R2: 0.9852163376957198


In [14]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())
])

param_grid = {
    "model__alpha": [0.01, 0.1, 1, 10, 100]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)

print("Best alpha:", grid.best_params_)
print("Test R2:", r2_score(y_test, grid.predict(X_test)))


Best alpha: {'model__alpha': 0.01}
Test R2: 0.9852164123437097


In [15]:
from sklearn.linear_model import Lasso

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(max_iter=10000))
])

param_grid = {
    "model__alpha": [0.001, 0.01, 0.1, 1]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)

print("Best alpha:", grid.best_params_)
print("Non-zero features:", np.sum(grid.best_estimator_.named_steps["model"].coef_ != 0))


Best alpha: {'model__alpha': 0.1}
Non-zero features: 8


In [16]:
from sklearn.linear_model import ElasticNet

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(max_iter=10000))
])

param_grid = {
    "model__alpha": [0.001, 0.01, 0.1, 1],
    "model__l1_ratio": [0.1, 0.5, 0.9]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)


Best params: {'model__alpha': 0.01, 'model__l1_ratio': 0.9}


In [17]:
from sklearn.preprocessing import PolynomialFeatures

pipe = Pipeline([
    ("poly", PolynomialFeatures()),
    ("scaler", StandardScaler()),
    ("model", Ridge())
])

param_grid = {
    "poly__degree": [1, 2, 3],
    "model__alpha": [0.1, 1, 10]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)


Best params: {'model__alpha': 0.1, 'poly__degree': 1}


In [18]:
from sklearn.linear_model import SGDRegressor

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(max_iter=2000, tol=1e-3))
])

param_grid = {
    "model__alpha": [1e-4, 1e-3, 1e-2],
    "model__penalty": ["l2", "l1", "elasticnet"],
    "model__eta0": [0.001, 0.01],
    "model__learning_rate": ["constant", "optimal"]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)


Best params: {'model__alpha': 0.001, 'model__eta0': 0.01, 'model__learning_rate': 'optimal', 'model__penalty': 'l2'}


In [19]:
from sklearn.metrics import mean_squared_error, r2_score

results = []

models = {
    "LinearRegression": model,
    "Ridge (best)": grid.best_estimator_ if isinstance(grid.best_estimator_.named_steps["model"], Ridge) else None,
}


In [20]:
ridge_grid = grid   # после Ridge
lasso_grid = grid   # после Lasso
enet_grid  = grid   # после ElasticNet
poly_grid  = grid   # после Polynomial
sgd_grid   = grid   # после SGD

In [21]:
models = {
    "LinearRegression": LinearRegression().fit(X_train, y_train),
    "Ridge": ridge_grid.best_estimator_,
    "Lasso": lasso_grid.best_estimator_,
    "ElasticNet": enet_grid.best_estimator_,
    "PolynomialRidge": poly_grid.best_estimator_,
    "SGDRegressor": sgd_grid.best_estimator_,
}

print(f"{'Model':<20} {'RMSE':<12} {'R2':<10}")

for name, m in models.items():
    y_pred = m.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{name:<20} {rmse:<12.3f} {r2:<10.3f}")


Model                RMSE         R2        
LinearRegression     15.259       0.985     
Ridge                17.790       0.980     
Lasso                17.790       0.980     
ElasticNet           17.790       0.980     
PolynomialRidge      17.790       0.980     
SGDRegressor         17.790       0.980     
